<a href="https://colab.research.google.com/github/yota-nagai/combinatorial-optimization-problems/blob/main/13_%E6%9C%80%E5%B0%8F%E5%B8%B0%E9%82%84%E8%BE%BA%E9%9B%86%E5%90%88%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q amplify
from amplify import VariableGenerator, FixstarsClient, solve, Solver
from datetime import timedelta

client = FixstarsClient()
client.token = "******************"  #アクセストークンを入力
client.parameters.timeout = timedelta(milliseconds=2000)  # タイムアウトは 2000 ms


#################### 最小帰還辺集合問題 ####################


"""
ダメ
https://amplify.fixstars.com/ja/techresources/research/ising-model-formulation/minimum-feedback-edge-set/
この通りに書いたつもりだがうまく動かない
モデルが複雑で、ソルバーが局所解にはまりやすく最適解にたどり着きづらい？
エネルギー関数の谷の地形がデコボコで険しい


N,M=map(int,input().split())  #頂点数、辺の数
edge=[]  #辺リスト（有向グラフ）
for _ in range(M):
  u,v=map(int,input().split())
  u-=1
  v-=1
  edge.append([u,v])

#バイナリ変数の生成
gen=VariableGenerator()
x_vi=gen.array("Binary",shape=(N,N))  #頂点vが高さiであるかどうか
y_uv=gen.array("Binary",shape=(N,N))  #辺(u,v)を残すか取り除くか

# x_uvi=gen.array("Binary",shape=(N,N,N))  y_uv=1（残す） かつ頂点uの高さがiであるとき1
# x_uviはx_uvi[u][v][i]=y_uv[u][v]*x_vi[u][i]をみたすものなので、上のように独立に生成してはダメ

#制約項①　各頂点vについて高さiがただ一つ存在
f1=0
for v in range(N):
  cnt=0
  for i in range(N):
    cnt+=x_vi[v][i]
  f1+=(1-cnt)**2

#制約項②　各辺(u,v)について、(u,v)を残すときに限り高さiがただ1つ存在し、取り除くときは高さを持たない
f2=0
for u,v in edge:
  cnt=0
  for i in range(N):
    cnt+=y_uv[u][v]*x_vi[u][i]
  f2+=(y_uv[u][v]-cnt)**2

#制約項③　高さをもつすべての辺について、低い方から高い方へ向かう（サイクルの禁止）
f3=0
for u,v in edge:
  for i in range(N):
    cnt=0
    for j in range(i+1,N):
      cnt+=x_vi[v][j]
    f3+=(y_uv[u][v]*x_vi[u][i])*(2-x_vi[u][i]-cnt)**2

#目的関数項　取り除く辺の数を最小化
f4=0
for u,v in edge:
  f4+=(1-y_uv[u][v])

#ペナルティ係数の設定
A,B=map(int,input().split())
f=A*(f1+f2+f3)+B*f4


#ソルバー実行、最適解の取得
result=solve(f,client)
objective=result.best.objective
values=result.best.values

print(objective)

"""


################################################

"""
こっちならよさそう
シンプルにした
エネルギー関数の谷の地形がなだらかでシンプルな坂道--->ソルバーが谷底を見つけやすい

"""

N,M=map(int,input().split())  #頂点数、辺の数
edge=[]  #辺リスト（有向グラフ）
for _ in range(M):
  u,v=map(int,input().split())
  u-=1
  v-=1
  edge.append([u,v])

#バイナリ変数の生成
gen=VariableGenerator()
x_vi=gen.array("Binary",shape=(N,N))  #頂点vが高さiであるかどうか
y_uv=gen.array("Binary",shape=(N,N))  #辺(u,v)を残すか取り除くか
# x_uvi=gen.array("Binary",shape=(N,N,N))  y_uv=1（残す） かつ頂点uの高さがiであるとき1
# x_uviはx_uvi[u][v][i]=y_uv[u][v]*x_vi[u][i]をみたすものなので、上のように独立に生成してはダメ

#制約項①　各頂点は、ただ一つの高さを持つ
f1=0
for v in range(N):
    f1+=(sum(x_vi[v])-1)**2

#制約項②　辺(u,v)を残すなら、高さがheight(u)<height(v)でなきゃダメ
f2=0
for u,v in edge:
    # height(u)>=height(v)を検出
    flag=0
    for i in range(N):
      for j in range(i,N):
        flag+=x_vi[u][j]*x_vi[v][i]
    # 辺を残し(y_uv=1)、かつheight(u)>=height(v)であるときにペナルティ
    f2+=y_uv[u][v]*flag

#目的関数項　取り除く辺の数を最小化
f3=0
for u,v in edge:
    f3+=(1-y_uv[u][v])

#ペナルティ係数の設定
A,B=map(int,input().split())
f=A*(f1+f2)+B*f3


# ソルバー実行、最適解の取得
result=solve(f,client)
objective=result.best.objective
values=result.best.values

#エネルギー関数を最小にする、辺の取り除き方
print(f"エネルギー関数の最小値 = {objective}")
y_uv_values=y_uv.evaluate(values)
eliminated=[]
for u,v in edge:
  if y_uv_values[u][v]==0:
    eliminated.append((u+1,v+1))
print(f"取り除く辺：{eliminated}")
